In [ ]:
!pip install jq -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 10.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

drive.mount("gdrive")


Mounted at gdrive


In [ ]:
!pip install langchain -q
!pip install langchain-core -q
!pip install langchain-community -q
!pip install -q langchain_gigachat


from langchain_community.document_loaders import JSONLoader


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00


In [ ]:
import pickle

with open("/content/gdrive/MyDrive/Studying/events.pkl", "rb") as f:
    data = pickle.load(f)


In [ ]:
def parse_data(data):
    news_data = {
        "type": [],
        "guid": [],
        "title": [],
        "content": [],
        "link": [],
        "pub_date": [],
        "topic": [],
        "sentiment": []
    }

    tweet_data = {
        "type": [],
        "tweet_id": [],
        "url": [],
        "tweet_created_at": [],
        "tweet": [],
        "user_name": [],
        "user_handle": [],
        "sentiment": []
    }



    for data_ent in data:
        if data_ent["type"] == "news":
            news_data["type"].append(data_ent["type"])
            for key in data_ent["data"].keys():
                if key not in ["entities"]:
                    news_data[key].append(data_ent["data"][key])
        else:
            tweet_data["type"].append(data_ent["type"])
            for key in data_ent["data"].keys():
                tweet_data[key].append(data_ent["data"][key])

    assert len(tweet_data["type"]) + len(news_data["type"]) == len(data)

    return {
        "news": news_data,
        "tweets": tweet_data
    }


In [ ]:
data = parse_data(data)


In [ ]:
data[0]

{'type': 'news',
 'data': {'guid': 'QTCX3BSV4FGYVNQWN6GPR5BEQA',
  'title': 'Tether CEO Ardoino Says He Expects U.S. Will Catch Up in Crypto Regulation',
  'content': 'Tether CEO Paolo Ardoino told a U.S. crowd on Tuesday that he\'s counting on sensible crypto regulations coming soon from the U.S. "There is no place like the U.S.," he said via video link at DC Fintech Week in Washington.',
  'link': 'https://www.coindesk.com/policy/2024/10/22/tether-ceo-ardoino-says-he-expects-us-will-catch-up-in-crypto-regulation/?utm_medium=referral&utm_source=rss&utm_campaign=headlines',
  'pub_date': datetime.datetime(2024, 10, 22, 18, 23, 41, tzinfo=datetime.timezone.utc),
  'topic': 'Regulation and Compliance',
  'sentiment': 'positive',
  'entities': ['Tether',
   'Paolo Ardoino',
   'U.S.',
   'DC Fintech Week',
   'Washington']}}

In [ ]:
data[-1]

{'type': 'tweet',
 'data': {'tweet_id': '1858918984919204351',
  'tweet_created_at': datetime.datetime(2024, 11, 19, 17, 3, 18, 216000, tzinfo=datetime.timezone.utc),
  'tweet': 'cbBTC is live on @arbitrum. cbBTC is an ERC-20 token that is backed 1:1 by Bitcoin (BTC) held by Coinbase. It is natively available on Arbitrum and securely accessible to more users across the Ethereum ecosystem. Start using cbBTC on day one ↓',
  'url': 'https://twitter.com/coinbase/status/1858918984919204351',
  'user_name': 'Coinbase 🛡️',
  'user_handle': 'coinbase',
  'sentiment': 'positive'}}

In [ ]:
import pandas as pd

news_data = pd.DataFrame(data["news"])
tweets_data = pd.DataFrame(data["tweets"])


In [ ]:
tweets_data.to_csv(
    "/content/gdrive/MyDrive/Studying/tweets_data.csv", index=False
)


In [ ]:
news_data.to_csv(
    "/content/gdrive/MyDrive/Studying/news_data.csv", index=False
)


In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

news_loader = CSVLoader(file_path="/content/gdrive/MyDrive/Studying/news_data.csv")
tweets_loader = CSVLoader(file_path="/content/gdrive/MyDrive/Studying/tweets_data.csv")

news_docs = news_loader.load()
tweets_docs = tweets_loader.load()



In [ ]:
assert (len(news_docs) + len(tweets_docs)) == (len(data["news"]["type"]) + len(data["tweets"]["type"]))


In [ ]:
print(news_docs[0].page_content[:1000])
print()
print(tweets_docs[0].page_content[:1000])

type: news
guid: QTCX3BSV4FGYVNQWN6GPR5BEQA
title: Tether CEO Ardoino Says He Expects U.S. Will Catch Up in Crypto Regulation
content: Tether CEO Paolo Ardoino told a U.S. crowd on Tuesday that he's counting on sensible crypto regulations coming soon from the U.S. "There is no place like the U.S.," he said via video link at DC Fintech Week in Washington.
link: https://www.coindesk.com/policy/2024/10/22/tether-ceo-ardoino-says-he-expects-us-will-catch-up-in-crypto-regulation/?utm_medium=referral&utm_source=rss&utm_campaign=headlines
pub_date: 2024-10-22 18:23:41+00:00
topic: Regulation and Compliance
sentiment: positive

type: tweet
tweet_id: 1848796310335357145
url: https://twitter.com/TheBlock__/status/1848796310335357145
tweet_created_at: 2024-10-22 18:39:24.299000+00:00
tweet: Notcoin founder says Binance and ‘poor guys from nowhere' are chasing success with Telegram games
user_name: The Block
user_handle: TheBlock__
sentiment: negative


In [ ]:
news_length = [len(news_docs[i].page_content) for i in range(len(news_docs))]
print(f"Max length news: {max(news_length)}")

tweets_length = [len(tweets_docs[i].page_content) for i in range(len(tweets_docs))]
print(f"Max length tweets: {max(tweets_length)}")


Max length news: 871
Max length tweets: 2093


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    add_start_index=True,
)

news_all_splits = text_splitter.split_documents(news_docs)
tweets_all_splits = text_splitter.split_documents(tweets_docs)

print(f"Split news into {len(news_all_splits)} sub-documents.")
print(f"Split tweets into {len(tweets_all_splits)} sub-documents.")


Split news into 1085 sub-documents.
Split tweets into 929 sub-documents.


In [ ]:
from tqdm.autonotebook import tqdm, trange

from langchain_core.vectorstores import InMemoryVectorStore

from sentence_transformers import SentenceTransformer

class EmbeddingModelWrapper:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()

    def embed_query(self, text):
        return self.model.encode([text])[0].tolist()


embedding_wrapper = EmbeddingModelWrapper("all-MiniLM-L6-v2")

vector_store = InMemoryVectorStore(embedding=embedding_wrapper)
news_docs_ids = vector_store.add_documents(documents=news_all_splits)
tweets_docs_ids = vector_store.add_documents(documents=tweets_all_splits)

print(news_docs_ids[:3])


['9fa3ff13-265d-4e14-9916-0d5cdf3deadf', '85530795-ef61-48f1-a09a-5663ef8e60c4', 'e73a01f3-c768-4f63-b57e-e1876140c2ec']


In [ ]:
from langchain import hub

prompt_template = hub.pull("rlm/rag-prompt")

example_messages = prompt_template.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


In [ ]:
print(example_messages[0].content)


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


In [ ]:
import os
from getpass import getpass

from langchain_gigachat import GigaChat

if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass()

llm = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")


··········


In [ ]:
question = "Give me top 10 best cryptocurrence on 2024?"

retrieved_docs = vector_store.similarity_search(question)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

prompt = prompt_template.invoke({"question": question, "context": docs_content})
answer = llm.invoke(prompt)


In [ ]:
print(prompt.messages[0].content)


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: Give me top 10 best cryptocurrence on 2024? 
Context: type: news
guid: RBS6MB7REZGGJBPYGZYFI5XKHE
title: The Real Winner of the 2024 Elections: The Crypto Industry
content: The 2024 elections marked a significant turning point for the crypto industry, with a pro-crypto president-elect advocating for the U.S. as the "crypto capital of the planet," paving the way for a shift from regulatory enforcement to a clearer, more predictable regulatory framework that will facilitate mainstream adoption and innovation in the sector, says Christopher Perkins.

type: news
guid: ZRKFPCJQFFAUXJSYRYHHLOAIPY
title: The Many Ways Crypto Won in This Election
content: The dust is starting to clear on the election and no one won bigger than crypto. Aubrey Strobel 

In [ ]:
print(answer.content)


I don't know the top 10 best cryptocurrencies for 2024. Predicting future rankings is complex and subject to market dynamics beyond any static context. Please consult up-to-date resources or experts in the field for this information.
